#### Import all modules:
- **preprocess:** includes utility functions such as contrast stretching, image type conversion, and water index computation  
- **singularity_index:** extracts curvilinear structures  
- **delineate:** delineate rivers and estimate their widths  
- **georef:** loads and saves georeferenced files  
- **visualization:** includes visualization functions  

In [1]:
import cv2
from rivamap import preprocess, singularity_index, delineate, georef, visualization

ModuleNotFoundError: No module named 'cv2'

Read bands 3 and 6 of an example Landsat 8 image.
You can download the example images from AWS:

- http://landsat-pds.s3.amazonaws.com/L8/138/045/LC81380452015067LGN00/LC81380452015067LGN00_B3.TIF
- http://landsat-pds.s3.amazonaws.com/L8/138/045/LC81380452015067LGN00/LC81380452015067LGN00_B6.TIF


In [ ]:
B3 = cv2.imread("../API/downloads/LC08_L2SP_149036_20240210_20240213_02_T1/L2SR_LC08_L2SP_149036_20240210_20240213_02_T1_SR_B3.TIF", cv2.IMREAD_UNCHANGED)
B6 = cv2.imread("../API/downloads/LC08_L2SP_149036_20240210_20240213_02_T1/L2SR_LC08_L2SP_149036_20240210_20240213_02_T1_SR_B6.TIF", cv2.IMREAD_UNCHANGED)

Compute the modified normalized difference water index of the input and save the result if needed.
You can do this also on Google Earth Engine.

In [ ]:
I1 = preprocess.mndwi(B3, B6)
cv2.imwrite("mndwi.TIF", cv2.normalize(I1, None, 0, 255, cv2.NORM_MINMAX))

Create the filters that are needed to compute the multiscale singularity index and apply the index to extract curvilinear structures from the input image. The singularity index function returns the overall singularity index response, width estimates, and channel orientation for each pixel whether or not they are river centerlines. We will find the river centerlines in the next step. You can save or view the overall singularity index response if needed.

In [ ]:
filters = singularity_index.SingularityIndexFilters()
psi, widthMap, orient = singularity_index.applyMMSI(I1, filters)
cv2.imwrite("psi.TIF", cv2.normalize(psi, None, 0, 255, cv2.NORM_MINMAX))

Extract and threshold centerlines to delineate rivers.

In [ ]:
nms = delineate.extractCenterlines(orient, psi)
centerlines = delineate.thresholdCenterlines(nms)
cv2.imwrite("nms.TIF", cv2.normalize(nms, None, 0, 255, cv2.NORM_MINMAX))

Generate a map of the extracted channels. The functions draw lines orthogonal to channel orientations to visualize width. The raster map function runs faster than the vector map function.

In [ ]:
raster = visualization.generateRasterMap(centerlines, orient, widthMap)
cv2.imwrite("rasterMap.TIF", cv2.normalize(raster, None, 0, 255, cv2.NORM_MINMAX))
visualization.generateVectorMap(centerlines, orient, widthMap, saveDest = "vector.pdf")

Create a quiver plot showing the magnitude and direction of channels. This function runs very slow for large input images.

In [ ]:
visualization.quiverPlot(psi, orient, saveDest = "quiver.pdf")

Save the results as georeferenced files.

In [ ]:
# An example of exporting a geotiff file
gm = georef.loadGeoMetadata("LC81380452015067LGN00_B3.TIF")
psi = preprocess.contrastStretch(raster)
psi = preprocess.double2im(raster, 'uint16')
georef.saveAsGeoTiff(gm, raster, "raster_geotagged.TIF")

# Export the (coordinate, width) pairs to a comma separated text file
georef.exportCSVfile(centerlines, widthMap, gm, "results.csv")
# Export line segments to a shapefile
georef.exportShapeFile(centerlines, widthMap, gm, "results")